In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import stats

import pyfstat
from pyfstat.helper_functions import get_sft_as_arrays

from IPython.display import Image, display

plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 20

In [ ]:
# Setup Writer
writer_kwargs ={"label": "single_detector_gaussian_noise",
                "outdir": "Pyfstat_example_data",
                "tstart": 1238166018,     # Starting time of the observating [GPS time]
                "duration": 120 * 86400,  # Duration [seconds]
                "detectors": "H1",        # Detector to simulate, in this case LIGO Hanford
                "sqrtSX": 1e-23,          # Single-sided Amplitude Spectral Density of the noise
                "Tsft": 1800,             # Fourier transform time duration
                "SFTWindowType": "tukey", # Window function to compute short Fourier transforms
                "SFTWindowBeta": 0.01,    # Parameter associated to the window function
               }

signal_parameters = {
    "F0": 100.,
    "F1": -1e-9,
    "Alpha": 0.5,
    "Delta": 0.,
    "h0": .5e-23,
    "cosi": 1,
    "psi": 0.,
    "phi": 0.
}

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)
writer.make_data()
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 10))

for ax in axs:
    ax.set(xlabel="Days since the start of O3", ylabel="Frequency [Hz]")

time_in_days = (timestamps["H1"] - timestamps["H1"][0]) / 86400

axs[0].set_title("SFT Real part")
c = axs[0].pcolormesh(time_in_days, frequency, fourier_data["H1"].real,
                      norm=colors.CenteredNorm())
fig.colorbar(c, ax=axs[0], orientation="horizontal", label="Fourier Amplitude")

axs[1].set_title("SFT Imaginary part")
c = axs[1].pcolormesh(time_in_days, frequency, fourier_data["H1"].imag,
                      norm=colors.CenteredNorm())

fig.colorbar(c, ax=axs[1], orientation="horizontal", label="Fourier Amplitude")

In [ ]:
signal_parameters["h0"] = writer_kwargs["sqrtSX"] / 10.

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)

# Create SFTs
writer.make_data()

frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 10))

for ax in axs:
    ax.set(xlabel="Days since the start of O3", ylabel="Frequency [Hz]")

time_in_days = (timestamps["H1"] - timestamps["H1"][0]) / 86400

axs[0].set_title("SFT Real part")
c = axs[0].pcolormesh(time_in_days, frequency, fourier_data["H1"].real,
                      norm=colors.CenteredNorm())
fig.colorbar(c, ax=axs[0], orientation="horizontal", label="Fourier Amplitude")

axs[1].set_title("SFT Imaginary part")
c = axs[1].pcolormesh(time_in_days, frequency, fourier_data["H1"].imag,
                      norm=colors.CenteredNorm())

fig.colorbar(c, ax=axs[1], orientation="horizontal", label="Fourier Amplitude")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 10))

for ax in axs:
    ax.set(xlabel="Days since the start of O3", ylabel="Frequency [Hz]")

time_in_days = (timestamps["H1"] - timestamps["H1"][0]) / 86400
     
axs[0].set_title("SFT absolute value")
c = axs[0].pcolorfast(time_in_days, frequency, np.absolute(fourier_data["H1"]))
fig.colorbar(c, ax=axs[0], orientation="horizontal", label="Value")

axs[1].set_title("SFT Phase")
c = axs[1].pcolorfast(time_in_days, frequency, np.angle(fourier_data["H1"]),
                      norm=colors.CenteredNorm())
fig.colorbar(c, ax=axs[1], orientation="horizontal", label="Value")

In [ ]:
theta_prior = {
    "F0": {
        "type": "unif",
        "lower": signal_parameters["F0"] - 1e-5,
        "upper": signal_parameters["F0"] + 1e-5,
    },
    "F1": {
        "type": "unif",
        "lower": signal_parameters["F1"] - 1e-11,
        "upper": signal_parameters["F1"] + 1e-11,
    },
    "Alpha": {
        "type": "unif",
        "lower": signal_parameters["Alpha"] - 1e-3,
        "upper": signal_parameters["Alpha"] + 1e-3,
    },
    "Delta": {
        "type": "unif",
        "lower": signal_parameters["Delta"] - 1e-3,
        "upper": signal_parameters["Delta"] + 1e-3,
    },
    "F2": 0.
}

mcmc = pyfstat.MCMCSearch(
    label=writer.label,
    outdir=writer.outdir,
    sftfilepattern=writer.sftfilepath,
    theta_prior=theta_prior,
    tref=writer.tref,
    nsteps=[100, 100],
    nwalkers=100,
    ntemps=3,
)

mcmc.run()
mcmc.plot_corner(add_prior=True, truths=signal_parameters)

In [ ]:
display(Image(filename=f"{mcmc.outdir}/{mcmc.label}_corner.png"))